In [1]:
import rasterio
import numpy as np
import os
# import re
import numpy as np

import json
import requests
import sys
import time
import argparse
import datetime
import json
# import pandas as pd
# from rasterio import Env
# from m2m_api.api import M2M

top_dir = r'C:\Users\12078\Downloads\l8cloudmasks'

with open('m2m_credentials.json') as f:
    credentials = json.load(f)




In [2]:
# send http request
def sendRequest(url, data, apiKey = None):  
    json_data = json.dumps(data)
    
    if apiKey == None:
        response = requests.post(url, json_data)
    else:
        headers = {'X-Auth-Token': apiKey}              
        response = requests.post(url, json_data, headers = headers)    
    
    try:
      httpStatusCode = response.status_code 
      if response == None:
          print("No output from service")
          sys.exit()
      output = json.loads(response.text)	
      if output['errorCode'] != None:
          print(output['errorCode'], "- ", output['errorMessage'])
          sys.exit()
      if  httpStatusCode == 404:
          print("404 Not Found")
          sys.exit()
      elif httpStatusCode == 401: 
          print("401 Unauthorized")
          sys.exit()
      elif httpStatusCode == 400:
          print("Error Code", httpStatusCode)
          sys.exit()
    except Exception as e: 
          response.close()
          print(e)
          sys.exit()
    response.close()
    
    return output['data']

In [3]:
# Login to the M2M API
serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
payload = {"username": credentials['username'], "password": credentials['password']}
apiKey = sendRequest(serviceUrl + "login", payload)

In [4]:
def download_scenes(dataset, serviceUrl, apiKey, sceneIds, downloadDir):
    # Find the download options for these scenes
    # NOTE :: Remember the scene list cannot exceed 50,000 items!
    payload = {'datasetName': dataset['datasetAlias'], 'entityIds': sceneIds}
    downloadOptions = sendRequest(serviceUrl + "download-options", payload, apiKey)

    # Aggregate a list of available products
    downloads = []
    for product in downloadOptions:
        # Make sure the product is available for this scene
        if product['available'] == True:
            downloads.append({'entityId': product['entityId'], 'productId': product['id']})

    # Did we find products?
    if downloads:
        requestedDownloadsCount = len(downloads)
        # set a label for the download request
        label = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") # Customized label using date time
        payload = {'downloads': downloads, 'label': label}
        # Call the download to get the direct download urls
        requestResults = sendRequest(serviceUrl + "download-request", payload, apiKey)

        # PreparingDownloads has a valid link that can be used but data may not be immediately available
        # Call the download-retrieve method to get download that is available for immediate download
        if requestResults['preparingDownloads'] != None and len(requestResults['preparingDownloads']) > 0:
            payload = {'label': label}
            moreDownloadUrls = sendRequest(serviceUrl + "download-retrieve", payload, apiKey)

            downloadIds = []
            for download in moreDownloadUrls['available']:
                if str(download['downloadId']) in requestResults['newRecords'] or str(download['downloadId']) in requestResults['duplicateProducts']:
                    downloadIds.append(download['downloadId'])
                    print("DOWNLOAD: " + download['url'])

            for download in moreDownloadUrls['requested']:
                if str(download['downloadId']) in requestResults['newRecords'] or str(download['downloadId']) in requestResults['duplicateProducts']:
                    downloadIds.append(download['downloadId'])
                    print("DOWNLOAD: " + download['url'])

            # Didn't get all of the requested downloads, call the download-retrieve method again probably after 30 seconds
            while len(downloadIds) < (requestedDownloadsCount - len(requestResults['failed'])):
                preparingDownloads = requestedDownloadsCount - len(downloadIds) - len(requestResults['failed'])
                print("\n", preparingDownloads, "downloads are not available. Waiting for 30 seconds.\n")
                time.sleep(30)
                print("Trying to retrieve data\n")
                moreDownloadUrls = sendRequest(serviceUrl + "download-retrieve", payload, apiKey)
                for download in moreDownloadUrls['available']:
                    if download['downloadId'] not in downloadIds and (str(download['downloadId']) in requestResults['newRecords'] or str(download['downloadId']) in requestResults['duplicateProducts']):
                        downloadIds.append(download['downloadId'])
                        print("DOWNLOAD: " + download['url'])

        else:
            # Get all available downloads
            for i, download in enumerate(requestResults['availableDownloads']):
                # TODO :: Implement a downloading routine
                print("DOWNLOAD: " + download['url'])
                download_url = download['url']

                # Specify the download directory and filename

                filename = f'download_{i}' + '.tar.gz'

                # Create the full file path
                file_path = os.path.join(downloadDir, filename)

                # Download the file
                response = requests.get(download_url)

                # Write the file
                with open(file_path, 'wb') as f:
                    f.write(response.content)

        print("\nAll downloads are available to download.\n")
    else:
        print("Search found no results.\n")


In [5]:
# Define dataset
datasetName = 'landsat_ot_c2_l1'
payload = {'datasetName': datasetName, 'maxResults': 10}
datasets = sendRequest(serviceUrl + "dataset-search", payload, apiKey)
dataset = datasets[0]

# Define scene IDs
payload = {'datasetName' : dataset['datasetAlias']}
# Now I need to run a scene search to find data to download
print("Searching scenes...\n\n")   
scenes = sendRequest(serviceUrl + "scene-search", payload, apiKey)
# Did we find anything?
if scenes['recordsReturned'] > 0:
    # Aggregate a list of scene ids
    sceneIds = []
    for result in scenes['results']:
        # Add this scene to the list I would like to download
        sceneIds.append(result['entityId'])

# Define download directory
downloadDir = os.path.join(top_dir, 'usgsdownloads')

download_scenes(dataset, serviceUrl, apiKey, sceneIds, downloadDir)

Searching scenes...


DOWNLOAD: https://dds.cr.usgs.gov/download/eyJpZCI6NTg1MDg3NzUxLCJjb250YWN0SWQiOjI3MDgyNzU5fQ==
DOWNLOAD: https://landsatlook.usgs.gov/gen-browse?size=frb&type=refl&product_id=LC08_L1GT_104028_20240318_20240318_02_RT&requestSignature=eyJjb250YWN0SWQiOjI3MDgyNzU5LCJkb3dubG9hZElkIjo1ODUxMzE0MDksImRhdGVHZW5lcmF0ZWQiOiIyMDI0LTAzLTE4VDE1OjU0OjUzLTA1OjAwIiwiaWQiOiJMQzA4X0wxR1RfMTA0MDI4XzIwMjQwMzE4XzIwMjQwMzE4XzAyX1JUIiwic2lnbmF0dXJlIjoiJDUkJGxFVERuS2VsVWttYmF5TmVKTGJiZGFBcXdSSER6dk85dUJhUmw2VWZ2b0IifQ==
DOWNLOAD: https://landsatlook.usgs.gov/gen-browse?size=frb&type=tir&product_id=LC08_L1GT_104028_20240318_20240318_02_RT&requestSignature=eyJjb250YWN0SWQiOjI3MDgyNzU5LCJkb3dubG9hZElkIjo1ODUxMzE0MTAsImRhdGVHZW5lcmF0ZWQiOiIyMDI0LTAzLTE4VDE1OjU0OjUzLTA1OjAwIiwiaWQiOiJMQzA4X0wxR1RfMTA0MDI4XzIwMjQwMzE4XzIwMjQwMzE4XzAyX1JUIiwic2lnbmF0dXJlIjoiJDUkJE9Ba0xIcFRsd1B2M3dLQk1HVGJJcUpEaFI0LmxuWWZVUmNqXC9seTIuOWQ1In0=
DOWNLOAD: https://landsatlook.usgs.gov/gen-browse?size=frb&type=qb&pr

KeyboardInterrupt: 

[{'abstractText': 'The USGS Earth Resources Observation and Science (EROS) Center archive holds data collected by the Landsat suite of satellites, beginning with Landsat 1 in 1972.  Landsat 8, launched February 11, 2013, is the latest Landsat sensor.', 'acquisitionStart': '2013-04-11', 'acquisitionEnd': None, 'catalogs': ['EE', 'GV'], 'collectionName': 'Landsat 8-9 OLI/TIRS C2 L1', 'collectionLongName': 'Landsat 8-9 Operational Land Imager and Thermal Infrared Sensor Collection 2 Level-1', 'datasetId': '5e81f14f59432a27', 'datasetAlias': 'landsat_ot_c2_l1', 'datasetCategoryName': 'Landsat Collection 2 Level-1', 'dataOwner': 'LSAA', 'dateUpdated': '2023-10-17 12:38:49.315571-05', 'doiNumber': 'https://doi.org//10.5066/P975CC9B', 'ingestFrequency': 'P1D', 'keywords': 'None,Visible Wavelengths,Earth Science,Infrared Wavelengths,Infrared Imagery,Imagery ,Earth Resources Observation and Science (EROS) Center,Coastal Aerosol ,Visible Imagery,U.S. Geological Survey (USGS),Thermal Infrared Sen

In [ ]:
test_folder_path = os.listdir(top_dir)[1]

files = os.listdir(os.path.join(top_dir, test_folder_path))

# data file has data in the name
data_files = [f for f in files if 'data' in f]
data_file = data_files[0]

# Open the data file
data_file_path = os.path.join(top_dir, test_folder_path, data_file)

# Open with rasterio
with rasterio.open(data_file_path) as src:
    data = src.read()

# Specify the folder path
folder_path = os.path.dirname(data_file_path)

# Iterate over the band numbers
for band_number in range(1, data.shape[0] + 1):
    # Get the band data
    band_data = data[band_number - 1]

    # Specify the band file path
    band_file_path = os.path.join(folder_path, f'band_{band_number}.tif')

    # Open the input image to get the parameters
    with rasterio.open(data_file_path) as src:
        profile = src.profile

    # Update the profile for the single-band image
    profile.update(count=1, dtype=band_data.dtype)

    # Write the single-band image
    with rasterio.open(band_file_path, 'w', **profile) as dst:
        dst.write(band_data, 1)


IndexError: list index out of range

In [ ]:
data_file_path

NameError: name 'data_file_path' is not defined

In [ ]:
# Go through the directory and get a list of all image names

# Get a list of all the image names
image_names = []

for filename in os.listdir(top_dir):
    if filename.endswith('_mtl.txt'):
        image_names.append(filename.split('_')[0])

# Create a folder for each image
for image_name in image_names:
    if not os.path.exists(os.path.join(top_dir, image_name)):
        os.makedirs(os.path.join(top_dir, image_name))

# Move the files into the appropriate folder
for filename in os.listdir(top_dir):
    file_path = os.path.join(top_dir, filename)
    if os.path.isfile(file_path):
        file_name = os.path.splitext(filename)[0]
        file_image_name = file_name.split('_')[0]
        new_file_path = os.path.join(top_dir, file_image_name, filename)
        os.rename(file_path, new_file_path)

In [ ]:
## OPEN ONE ## 
# Specify the folder you want to open
folder_path = os.listdir(top_dir)[0]

# Get a list of all files in the folder
files = os.listdir(os.path.join(top_dir, folder_path))

# data file has data in the name
data_files = [f for f in files if 'data' in f]
data_file = data_files[0]

# Open the data file
data_file_path = os.path.join(top_dir, folder_path, data_file)

# Open with rasterio
with rasterio.open(data_file_path) as src:
    data = src.read()

# Print the shape of the data
print(data.shape)

# Specify the folder you want to open
folder_path = os.listdir(top_dir)[0]

# Get a list of all files in the folder
files = os.listdir(os.path.join(top_dir, folder_path))

# data file has data in the name
data_files = [f for f in files if 'data' in f]
data_file = data_files[0]

# Open the data file
data_file_path = os.path.join(top_dir, folder_path, data_file)

# Open with rasterio
with rasterio.open(data_file_path) as src:
    data = src.read()



# Save as individual bands
b1_path = os.path.join(top_dir, folder_path, folder_path + '_B1.TIF')
b2_path = os.path.join(top_dir, folder_path, folder_path + '_B2.TIF')
b3_path = os.path.join(top_dir, folder_path, folder_path + '_B3.TIF')
b4_path = os.path.join(top_dir, folder_path, folder_path + '_B4.TIF')
b5_path = os.path.join(top_dir, folder_path, folder_path + '_B5.TIF')
b6_path = os.path.join(top_dir, folder_path, folder_path + '_B6.TIF')
b7_path = os.path.join(top_dir, folder_path, folder_path + '_B7.TIF')
b8_path = os.path.join(top_dir, folder_path, folder_path + '_B8.TIF')
b9_path = os.path.join(top_dir, folder_path, folder_path + '_B9.TIF')
b10_path = os.path.join(top_dir, folder_path, folder_path + '_B10.TIF')


# Specify the band numbers
band_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Iterate over the band numbers
# Specify the output file path
output_file_path = 'path/to/output/file.tif'

# Open the input image to get the parameters
with rasterio.open(data_file_path) as src:
    profile = src.profile

# Write the image with the same parameters
with rasterio.open(output_file_path, 'w', **profile) as dst:
    dst.write(data)



# # Delete the src file
# os.remove(data_file_path)



IndexError: list index out of range

In [ ]:
data_file_path

NameError: name 'data_file_path' is not defined

In [ ]:
top_dir = r'C:\Users\12078\Downloads\l8cloudmasks\sending'

In [ ]:
# Specify the top directory
top_dir = r'C:\Users\12078\Downloads\l8cloudmasks\sending'

# Get a list of all files and folders in the top directory
files = os.listdir(top_dir)

# Filter out the image names
image_names = [filename for filename in files if os.path.isdir(os.path.join(top_dir, filename))]

# Print the list of image names
print(image_names)

# download these images using the m2m api



['LC80010812013365LGN00', 'LC80020622013244LGN00', 'LC80050152014172LGN00', 'LC80050562014076LGN00', 'LC80150242014146LGN00', 'LC80190352014078LGN01', 'LC80200462014213LGN00', 'LC80250402013245LGN00', 'LC80250482014072LGN00', 'LC80310432013207LGN00', 'LC80320282013182LGN00', 'LC80340412013132LGN01', 'LC80380362013144LGN00', 'LC80390152013215LGN00', 'LC80430352013243LGN00', 'LC80470252014226LGN00', 'LC80500162014247LGN00', 'LC80630462013191LGN00', 'LC80750722014022LGN00', 'LC80750962014038LGN00', 'LC80760912014013LGN00', 'LC80800142014169LGN00', 'LC80900832013220LGN00', 'LC80910892014006LGN00', 'LC80940802014092LGN00', 'LC80980732014008LGN00', 'LC81020642014004LGN00', 'LC81060772014064LGN00', 'LC81110822013288LGN00', 'LC81130672013142LGN01', 'LC81140272013261LGN00', 'LC81140622013277LGN00', 'LC81180602014116LGN00', 'LC81220242014240LGN00', 'LC81230362013276LGN00', 'LC81250542014181LGN00', 'LC81250622013178LGN01', 'LC81280422014074LGN00', 'LC81330492014141LGN00', 'LC81340422013257LGN00',